## Review

Now, we have some understanding of a few things:

- How to customize the graph state schema
- How to define custom state reducers
- How to use multiple graph state schemas

## Goals

- We build a chatbot that has long-term memory.
- use advanced techniques to to work with messages.

In [7]:
# general imports
import sys
import os, getpass
from openai import OpenAI
from utils import *
from IPython.display import Image
from pprint import pprint

# Import langgraph modules to create and visualize the graph
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage 
from typing_extensions import TypedDict
from typing import Literal
from pydantic import BaseModel, Field

# Install the missing package
# %pip install langchain_anthropic
# %pip install langchain_groq
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool

In [2]:
messages = [AIMessage(f'So you said you were researching about Hindu mythology Ramayana.', name='AI')]
messages.append(HumanMessage(f'Yes, I am researching about Hindu mythology Ramayana. But I also want to understand the time it happened and what was its significance.', name='Human'))

In [3]:
# Let's make sure we have the API key set
load_dotenv()
GROQ_API_KEY = set_api_key('GROQ_API_KEY')
ANTHROPIC_API_KEY = set_api_key('ANTHROPIC_API_KEY')

API key found in .env file for GROQ_API_KEY
API key found in .env file for ANTHROPIC_API_KEY


In [4]:
ai = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY, temperature=0, max_retries=2)

In [5]:
result = ai.invoke(messages)
result.pretty_print()

================================== Ai Message ==================================

The Ramayana is an ancient Indian epic that has been a significant part of Hindu mythology for centuries. The story revolves around the life of Lord Rama, a king of Ayodhya, and his journey to rescue his wife Sita from the clutches of the demon king Ravana.

**Time Period:**
The exact time period of the Ramayana is not known, but it is believed to have occurred during the Treta Yuga, which is the second of the four yugas (ages) in Hindu mythology. According to traditional Hindu accounts, the Treta Yuga began around 1.7 million years ago and lasted for 1.2 million years. However, most scholars and historians estimate that the events of the Ramayana took place around 5000-7000 years ago, during the late Bronze Age or early Iron Age.

**Significance:**
The Ramayana has immense significance in Hindu mythology and culture. It is considered one of the most important epics in Hinduism, along with the Mahabharata

In [6]:
from langchain_core.messages import trim_messages

In [8]:
def chat_model_node(state:MessagesState):
    messages = trim_messages(
        state["messages"],
        max_token= 100,
        strategy = 'last',
        token_counter = ai.get_num_tokens,
        allow_partial=True
    )
    return {"messages": ai.invoke(messages)} 

In [9]:
builder = StateGraph(MessagesState)
builder.add_node("chat_model", chat_model_node)
builder.add_edge(START, "chat_model")
builder.add_edge("chat_model", END)
graph = builder.compile() 